In [105]:
#1st pass for NANO 120B NN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization

In [3]:
import numpy as np

In [101]:
TDS_1 = np.random.rand(1,300)
TDS_1 =np.transpose(TDS_1)
display(TDS_1)
print(TDS_1.shape)

array([[0.01293303],
       [0.31376082],
       [0.97031867],
       [0.55308881],
       [0.51290874],
       [0.05680984],
       [0.63228572],
       [0.42337275],
       [0.37164429],
       [0.14922935],
       [0.43487677],
       [0.2780989 ],
       [0.81150517],
       [0.92886034],
       [0.53595647],
       [0.7293878 ],
       [0.95395196],
       [0.98669588],
       [0.05209101],
       [0.66311152],
       [0.10122784],
       [0.78627824],
       [0.96745458],
       [0.70666881],
       [0.23380188],
       [0.12066756],
       [0.38927169],
       [0.94348528],
       [0.82335521],
       [0.23599657],
       [0.72715141],
       [0.33868559],
       [0.05762361],
       [0.1786753 ],
       [0.02895029],
       [0.17817784],
       [0.76114217],
       [0.20798457],
       [0.67428215],
       [0.03872573],
       [0.0760381 ],
       [0.1267044 ],
       [0.48131107],
       [0.86664507],
       [0.22265254],
       [0.19531242],
       [0.95637238],
       [0.005

(300, 1)


# Dense NN
Simplest form of a neural network.

In [126]:
#Dense: Simpliest form of a neural network.
#These hyperparameters are modeled after the DANN NN described in: https://link.springer.com/article/10.1007/s00521-020-04853-3
#Input shape is (1, 300), arrays from training_data will have to be transposed before input.

model = Sequential(name="sequantial1")
model.add(Dense(300, input_shape=(TDS_1.shape[1],TDS_1.shape[0]), name="fc1"))
model.add(Activation(activation="relu", name="relu1"))
model.add(Dense(512, name="fc2"))
model.add(Activation(activation="relu", name="relu2"))
model.add(Dense(1536, name="fc3"))
model.add(Activation(activation="relu", name="relu3"))
model.add(Dense(8, name="fc4"))
model.add(Activation(activation="sigmoid", name="sigmoid"))
model.summary()

Model: "sequantial1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (Dense)                 (None, 1, 300)            90300     
                                                                 
 relu1 (Activation)          (None, 1, 300)            0         
                                                                 
 fc2 (Dense)                 (None, 1, 512)            154112    
                                                                 
 relu2 (Activation)          (None, 1, 512)            0         
                                                                 
 fc3 (Dense)                 (None, 1, 1536)           787968    
                                                                 
 relu3 (Activation)          (None, 1, 1536)           0         
                                                                 
 fc4 (Dense)                 (None, 1, 8)              

In [ ]:
#An example of the CANN network described in https://link.springer.com/article/10.1007/s00521-020-04853-3
#Using TDS spectral data to predict hydrogen sensitivity parameter (HSP) in steels
#This model was made for 762 data points, and a single output, so will need adjustment for our purposes.
#MaxPooling layers take the output and "pool" the data together based on that stride's maximum value.

model = keras.models.Sequential(name="QuickTDS_CANN")
#Convolution Layers
model.add(layers.Conv1D(filters=1, kernel_size=30, strides=1, padding="same", activation="relu",\
                        input_shape=(TDS_1.shape[0], TDS_1.shape[1])))
model.add(layers.MaxPooling1D(pool_size=2, strides=None, padding="same", data_format="channels_last"))
model.add(layers.Conv1D(filters=100, kernel_size=30, strides=2, padding="same", activation="relu"))
model.add(layers.MaxPooling1D(pool_size=2, strides=None, padding="same", data_format="channels_last"))
model.add(layers.Conv1D(filters=160, kernel_size=30, strides=2, padding="same", activation="relu"))
model.add(layers.MaxPooling1D(pool_size=2, strides=None, padding="same", data_format="channels_last"))
model.add(layers.Conv1D(filters=160, kernel_size=30, strides=2, padding="same", activation="relu"))
model.add(layers.MaxPooling1D(pool_size=2, strides=None, padding="same", data_format="channels_last"))
model.add(layers.Conv1DTranspose(filters=1, kernel_size=30, padding="same", name = "output"))
model.summary()

Model: "QuickTDS_CANN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_321 (Conv1D)         (None, 300, 1)            61        
                                                                 
 max_pooling1d_22 (MaxPoolin  (None, 150, 1)           0         
 g1D)                                                            
                                                                 
 conv1d_322 (Conv1D)         (None, 75, 100)           3100      
                                                                 
 max_pooling1d_23 (MaxPoolin  (None, 38, 100)          0         
 g1D)                                                            
                                                                 
 conv1d_323 (Conv1D)         (None, 19, 160)           480160    
                                                                 
 max_pooling1d_24 (MaxPoolin  (None, 10, 160)        

In [103]:
#A basic CNN model. Input shape is set as TDS_1 shape aka (300, 1) for one TDS spectrum.
#A CNN downsamples data based on filter_size,\
# larger filter_size, the larger "area" of data is downsampled for each kernel.
#strides = number of data steps skipped from one kernel to the next.
#Strides were chosen by calculating what factors give a shape (8, ) with 300 datapoints. 
#"same" padding ensures the input and output of that layer is the same size.

model = keras.models.Sequential(name="QuickTDS_DeepClean")
#Convolution Layers
model.add(layers.Conv1D(filters=1, kernel_size=7, strides=1, padding="same", activation="tanh",\
                        input_shape=(TDS_1.shape[0], TDS_1.shape[1])))
model.add(layers.Conv1D(filters=8, kernel_size=7, strides=5, padding="same", activation="tanh"))
model.add(layers.Conv1D(filters=16, kernel_size=7, strides=2, padding="same", activation="tanh"))
model.add(layers.Conv1D(filters=32, kernel_size=7, strides=2, padding="same", activation="tanh"))
model.add(layers.Conv1D(filters=64, kernel_size=7, strides=2, padding="same", activation="tanh"))
model.add(layers.Conv1D(filters=1, kernel_size=7, padding="same", name = "output"))
model.summary()

Model: "QuickTDS_CNN"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_360 (Conv1D)         (None, 300, 1)            8         
                                                                 
 conv1d_361 (Conv1D)         (None, 60, 8)             64        
                                                                 
 conv1d_362 (Conv1D)         (None, 30, 16)            912       
                                                                 
 conv1d_363 (Conv1D)         (None, 15, 32)            3616      
                                                                 
 conv1d_364 (Conv1D)         (None, 8, 64)             14400     
                                                                 
 output (Conv1D)             (None, 8, 1)              449       
                                                                 
Total params: 19,449
Trainable params: 19,449
Non-trai